# Topic Modelling on Program Source Code
---
By Kishalay Banerjee, Dan Jones and Sam Harding

In [ ]:
import warnings
warnings.filterwarnings('ignore')  # 0y

In [ ]:
import pandas
import pickle
import math
import numpy
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from urllib.request import urlretrieve

In [ ]:
numpy.random.seed(0xC0FFEE) 

## Preparation

There are a number of files which are too large to store on GitHub. These are hosted on our server, and can be downloaded by running the following cell:

In [ ]:
files = [
    'full_lda_model.pickle',
    'full_tf.pickle',
    'full_tf_vectorizer.pickle',
    'full-dataset.csv.gz',
]

base_url = 'https://daniel.wilshirejones.com/private-uUX6IzfsRYLNiti4ZFmgv6U3dFInnq37r5YSQs46iejeB96q0MAy9Ko7hkgo/'
destination_directory = '../data/'

for file in files:
    url = base_url + file
    destination = destination_directory + file
    print("Downloading '{}'' to location '{}'".format(url, destination))
    urlretrieve(url, destination)

## Generating the Dataset

TODO: Import dataset.py and explain how it's used + what it does.

TODO: Add Sam's scrub function in here?

In [ ]:
minimal_dataset = pandas.read_csv("../data/dataset.csv.gz", header=None, names=['repo', 'language', 'documents'])
minimal_dataset.head()

In [ ]:
full_dataset = pandas.read_csv("../data/full-dataset.csv.gz", header=None, names=['repo', 'language',  'topics', 'documents'])

# Remove Github 'topics' since we don't use them in this analysis
full_dataset = full_dataset.drop(columns='topics')

full_dataset.head()

In [ ]:
test_dataset = pandas.read_csv("../data/test-dataset.csv.gz", header=None, names=['repo', 'language', 'topics', 'documents'])

# Remove Github 'topics' since we don't use them in this analysis
test_dataset = test_dataset.drop(columns='topics')

test_dataset.head()

To evaluate the mixture model, we must label each repository with it's percentage of each programming language:

In [ ]:
def calculate_language_percentages(group):
    total_python_length = 0
    total_r_length = 0
    total_javascript_length = 0
    
    for index, repo, language, document in group.itertuples():
        if language == 'python':
            total_python_length += len(document)
            
        if language == 'javascript':
            total_javascript_length += len(document)
            
        if language == 'r':
            total_r_length += len(document)
            
    total_length = total_python_length + total_r_length + total_javascript_length
            
    return pandas.Series([
        total_python_length/total_length,
        total_r_length/total_length,
        total_javascript_length/total_length,
    ])

In [ ]:
test_composition_actual = test_dataset.groupby(by='repo').apply(calculate_language_percentages)
test_composition_actual.columns = ['python', 'r', 'javascript']

Here are the programming language percentages for each of repository in our test dataset:

In [ ]:
test_composition_actual

In [ ]:
def concat_texts(group):
    [repo_id] = group['repo'].unique()
    combined = ' '.join(group['documents'])
    return combined

In [ ]:
combined_test_documents = test_dataset.groupby(by='repo').apply(concat_texts)

## Topic Modelling on Individual Source Files

Basically done, just need to copy it over. Maybe run on the bigger dataset?

TODO:
  - Copy work from documentation/daniel-jones.ipynb
  - Add visualisation with pyldavis

For our purposes, common words are important and rare words aren't. So we shouldn't use tf-idf as a metric, bag-of-words makes more sense. (TODO: Maybe: "Similarly, filter out words that don't occur very often").


In [ ]:
documents = minimal_dataset['documents']

In [ ]:
tf_vectorizer = CountVectorizer(stop_words=None)
tf = tf_vectorizer.fit_transform(documents)

with open('../data/minimal_lda_tf.pickle', 'wb') as f:
    pickle.dump(tf, f)
    
with open('../data/minimal_lda_tf_vectorizer.pickle', 'wb') as f:
    pickle.dump(tf_vectorizer, f)

We have four programming languages, try to use LDA to determine these four programming languages.

In [ ]:
number_of_languages = 4

lda = LatentDirichletAllocation(n_topics=number_of_languages,  n_jobs=1)
model = lda.fit(tf)

with open('../data/minimal_lda_model.pickle', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
with open('../data/minimal_lda_model.pickle', 'rb') as f:
    model = pickle.load(f)
    
with open('../data/minimal_lda_tf.pickle', 'rb') as f:
    tf = pickle.load(f)
    
with open('../data/minimal_lda_tf_vectorizer.pickle', 'rb') as f:
    tf_vectorizer = pickle.load(f)

In [ ]:
pyLDAvis.sklearn.prepare(model, tf, tf_vectorizer)

Try to do this on the full dataset:

In [ ]:
number_of_languages = 4
all_documents = full_dataset['documents']

full_tf_vectorizer = CountVectorizer(stop_words=None)
full_tf = full_tf_vectorizer.fit_transform(all_documents)
full_tf_feature_names = full_tf_vectorizer.get_feature_names()

full_lda = LatentDirichletAllocation(n_topics=number_of_languages,  n_jobs=1)
full_model = full_lda.fit(full_tf)

with open('../data/full_lda_model.pickle', 'wb') as f:
    pickle.dump(full_model, f)

with open('../data/full_tf.pickle', 'wb') as f:
    pickle.dump(full_tf, f)
    
with open('../data/full_tf_vectorizer.pickle', 'wb') as f:
    pickle.dump(full_tf_vectorizer, f)

In [ ]:
with open('../data/full_lda_model.pickle', 'rb') as f:
    full_model = pickle.load(f)

with open('../data/full_tf.pickle', 'rb') as f:
    full_tf = pickle.load(f)
    
with open('../data/full_tf_vectorizer.pickle', 'rb') as f:
    full_tf_vectorizer = pickle.load(f)

Visualize our new model. Note that the following code cell requires more than 8 GB of RAM to run.

In [ ]:
pyLDAvis.sklearn.prepare(full_model, full_tf, full_tf_vectorizer)

Reference: https://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/sklearn.ipynb

## Topic Modelling on Programming Language Mixtures
Keypoint: topics are programming languages, file with mixture of programming languages, identify which is which.

Applicability to cyber-security: identifying malware embedded within normal programs (shellcode).

### Combining Source Files per Repository

Currently our data set consists of one data point per file containing:
  1. ID of the Github repository the file belongs to.
  2. Programming language it is written in (identified by file extension).
  3. File contents
  
In this section, we extend our analysis from working on documents with one language per file, to a system where there is a mixture of languages inside each document. To do this, we combine all the files in each repository into a single data point.

In [ ]:
combined_documents = minimal_dataset.groupby(by='repo').apply(concat_texts)

In [ ]:
combined_documents.head()

### LDA Model 

In [ ]:
combined_test_documents.append(combined_documents)

In [ ]:
tf_vectorizer = CountVectorizer(stop_words=None)
tf_vectorizer.fit(combined_test_documents.append(combined_documents))

tf_train = tf_vectorizer.transform(combined_documents)
tf_test = tf_vectorizer.transform(combined_test_documents)

with open('../data/concatDocs_tf_vectorizer.pickle', 'wb') as f:
    pickle.dump(tf_vectorizer, f)
    
with open('../data/concatDocs_tf_train.pickle', 'wb') as f:
    pickle.dump(tf_train, f)
    
with open('../data/concatDocs_tf_test.pickle', 'wb') as f:
    pickle.dump(tf_test, f)

In [ ]:
number_of_languages = 4

lda = LatentDirichletAllocation(n_topics=number_of_languages,  n_jobs=1)
model = lda.fit(tf_train)

with open('../data/concatDocs_lda_model.pickle', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
with open('../data/concatDocs_lda_model.pickle', 'rb') as f:
    model = pickle.load(f)
    
with open('../data/concatDocs_tf_train.pickle', 'rb') as f:
    tf_train = pickle.load(f)
    
with open('../data/concatDocs_tf_vectorizer.pickle', 'rb') as f:
    tf_vectorizer = pickle.load(f)

with open('../data/concatDocs_tf_test.pickle', 'rb') as f:
    tf_test = pickle.load(f)

In [ ]:
pyLDAvis.sklearn.prepare(model, tf_train, tf_vectorizer)

In [ ]:
feature_map = dict(enumerate(tf_vectorizer.get_feature_names()))

In [ ]:
len(model.components_), len(model.components_[0]), len(feature_map)

In [ ]:
# importance_of_feature_in_topic = model.components_[<topic_index>][<feature_index>]

word_importance_per_topic = []

for topic_components in model.components_:
    word_importance = [
        (feature_map[feature_index], feature_importance) 
        for feature_index, feature_importance in enumerate(topic_components)
    ]
    word_importance = sorted(word_importance, key=lambda tup: tup[1], reverse=True)
    word_importance_per_topic.append(word_importance)

In [ ]:
# Note: visualise via heatmap?

most_importance_topic_1_keywords = [word for word, importance in word_importance_per_topic[0][0:40]]
most_importance_topic_2_keywords = [word for word, importance in word_importance_per_topic[1][0:40]]
most_importance_topic_3_keywords = [word for word, importance in word_importance_per_topic[2][0:40]]
most_importance_topic_4_keywords = [word for word, importance in word_importance_per_topic[3][0:40]]

In [ ]:
print(
    100*jaccard_index(javascript_keywords, most_importance_topic_1_keywords)/len(javascript_keywords),
    100*jaccard_index(javascript_keywords, most_importance_topic_2_keywords)/len(javascript_keywords),
    100*jaccard_index(javascript_keywords, most_importance_topic_3_keywords)/len(javascript_keywords),
    100*jaccard_index(javascript_keywords, most_importance_topic_4_keywords)/len(javascript_keywords),
)

print(
    100*jaccard_index(r_keywords, most_importance_topic_1_keywords)/len(r_keywords),
    100*jaccard_index(r_keywords, most_importance_topic_2_keywords)/len(r_keywords),
    100*jaccard_index(r_keywords, most_importance_topic_3_keywords)/len(r_keywords),
    100*jaccard_index(r_keywords, most_importance_topic_4_keywords)/len(r_keywords),
)

print(
    100*jaccard_index(python_keywords, most_importance_topic_1_keywords)/len(python_keywords),
    100*jaccard_index(python_keywords, most_importance_topic_2_keywords)/len(python_keywords),
    100*jaccard_index(python_keywords, most_importance_topic_3_keywords)/len(python_keywords),
    100*jaccard_index(python_keywords, most_importance_topic_4_keywords)/len(python_keywords),
) 

## Alternative Splitting Function

One potential problem we identified was with the method that sklearn uses to split up the program into "words". The base function searches the program for strings of two or more alphanumeric characters in a row, taking these as the words of a particular document. The issue here is that most code contains comments (which are heavy in alphanumeric characters that offer little to no help in detecting the programming language (or at least it is hard to detect the phrases that are helpful)), and also non-alphanumeric characters that might help to detect the language (for instance certain forms of brackets, colons, and other indicators). Therefore we set to design a new splitting function which would hopefully filter out non-useful information and leave us with a list of words that has more to say about the language being written.

In [ ]:
def trim(text):
    if type(text)==str:
        return text.lstrip().rstrip()
    elif type(text)==list:
        tram=[]
        for i in text:
            tram.append(i.lstrip().rstrip())
        return tram
    else:
        return "Not Correct Format"
def splat(text, splt):
    if type(text)==str:
        splot=str.split(text[2:-1], splt)
        return trim(splot)
    elif type(text)==list:
        splot=[]
        for i in text:
            for j in str.split(i,splt):
                splot.append(j)
        return trim(splot)
    else:
        return "Not Correct Format"
def scrub(string, inclComments=False):
    x=string
    x=splat(x,"\\n")
    if inclComments==False:
        x=[i for i in x if i[0:2]!="//"]
        x=[i for i in x if i[0:1]!="#"]
    x=splat(x,"(")
    x=splat(x,"[")
    x=splat(x,"{")
    x=splat(x," ")
    x=[i for i in x if i!=""]
    return x
def concatMe(a):
    return " ".join(a)

We then apply an almost identical method as before to run LDA on these new modified documents:

In [ ]:
combined_split_documents=combined_documents.apply(scrub).apply(concatMe)
combined_split_test_documents=combined_test_documents.apply(scrub).apply(concatMe)

In [ ]:
split_tf_vectorizer = CountVectorizer(stop_words=None, token_pattern="[^ ]*")
split_tf_vectorizer.fit(combined_split_test_documents.append(combined_split_documents))

split_tf_train = split_tf_vectorizer.transform(combined_split_documents)
split_tf_test = split_tf_vectorizer.transform(combined_split_test_documents)

with open('../data/concatDocs_split_tf_vectorizer.pickle', 'wb') as f:
    pickle.dump(split_tf_vectorizer, f)
    
with open('../data/concatDocs_split_tf_train.pickle', 'wb') as f:
    pickle.dump(split_tf_train, f)
    
with open('../data/concatDocs_split_tf_test.pickle', 'wb') as f:
    pickle.dump(split_tf_test, f)

In [ ]:
number_of_languages = 4

lda = LatentDirichletAllocation(n_topics=number_of_languages,  n_jobs=1)
model = lda.fit(split_tf_train)

with open('../data/concatDocs_split_lda_model.pickle', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
with open('../data/concatDocs_split_lda_model.pickle', 'rb') as f:
    model = pickle.load(f)
    
with open('../data/concatDocs_split_tf_train.pickle', 'rb') as f:
    tf_train = pickle.load(f)
    
with open('../data/concatDocs_split_tf_vectorizer.pickle', 'rb') as f:
    tf_vectorizer = pickle.load(f)

with open('../data/concatDocs_split_tf_test.pickle', 'rb') as f:
    tf_test = pickle.load(f)

In [ ]:
pyLDAvis.sklearn.prepare(model, tf_train, tf_vectorizer)

## Identifying Program Subjects and Themes

Hypothesis: Using tf-idf rather than bag-of-words as an input to LDA will prioritise rare words. In the case of source code, this means programming language keywords (an identifying feature of programming languages) are deprioritised, and so a more human idea of topics may emerge. 

We can use repo-list.json and the repo-ids to map the github topics/tags to each repo. Might be a small/easy task to compare against the programming langauge identification.

In [ ]:
documents = minimal_dataset['documents']

In [ ]:
tf_vectorizer = TfidfVectorizer(stop_words=None)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

with open('../data/tfidf_lda_tf.pickle', 'wb') as f:
    pickle.dump(tf, f)
    
with open('../data/tfidf_lda_tf_vectorizer.pickle', 'wb') as f:
    pickle.dump(tf_vectorizer, f)

We have four programming languages, try to use LDA to determine these four programming languages.

In [ ]:
number_of_themes = 3

lda = LatentDirichletAllocation(n_topics=number_of_themes,  n_jobs=1)
model = lda.fit(tf)

with open('../data/tfidf_lda_model.pickle', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
with open('../data/tfidf_lda_model.pickle', 'rb') as f:
    model = pickle.load(f)
    
with open('../data/tfidf_lda_tf.pickle', 'rb') as f:
    tf = pickle.load(f)
    
with open('../data/tfidf_lda_tf_vectorizer.pickle', 'rb') as f:
    tf_vectorizer = pickle.load(f)

In [ ]:
pyLDAvis.sklearn.prepare(model, tf, tf_vectorizer)

This still prioritises programming language keywords. One approach to solving this problem is to consider all keywords as "stopwords". First, gather a list of R, Python and Javascript keywords:

In [ ]:
import keyword

python_keywords = keyword.kwlist
python_keywords

R reserved words (sourced from the manual: https://stat.ethz.ch/R-manual/R-devel/library/base/html/Reserved.html)

In [ ]:
r_keywords = [
    "if", 
    "else", 
    "repeat",
    "while",
    "function", 
    "for",
    "in",
    "next",
    "break",
    "TRUE",
    "FALSE",
    "NULL", 
    "Inf", 
    "NaN",
    "NA",
    "NA_integer_",
    "NA_real_",
    "NA_complex_",
    "NA_character_", 
]

Javascript keywords and reserved words (source: https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Lexical_grammar#Keywords)

In [ ]:
javascript_keywords = [  # jaccard("ideal javascript topic", topic_i)
    "break",
    "case",
    "catch",
    "class",
    "const",
    "continue",
    "debugger",
    "default",
    "delete",
    "do",
    "else",
    "export",
    "extends",
    "finally",
    "for",
    "function",
    "if",
    "import",
    "in",
    "instanceof",
    "new",
    "return",
    "super",
    "switch",
    "this",
    "throw",
    "try",
    "typeof",
    "var",
    "void",
    "while",
    "with",
    "yield",
]

In [ ]:
documents = minimal_dataset['documents']

In [ ]:
tf_vectorizer = TfidfVectorizer(stop_words=javascript_keywords+python_keywords+r_keywords)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()
    
with open('../data/tfidf_lda_tf_ignore_keywords.pickle', 'wb') as f:
    pickle.dump(tf, f)
    
with open('../data/tfidf_lda_tf_vectorizer_ignore_keywords.pickle', 'wb') as f:
    pickle.dump(tf_vectorizer, f)

We have four programming languages, try to use LDA to determine these four programming languages.

In [ ]:
number_of_themes = 3

lda = LatentDirichletAllocation(n_topics=number_of_themes,  n_jobs=1)
model = lda.fit(tf)

with open('../data/tfidf_lda_model_ignore_keywords.pickle', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
with open('../data/tfidf_lda_model_ignore_keywords.pickle', 'rb') as f:
    model = pickle.load(f)
    
with open('../data/tfidf_lda_tf_ignore_keywords.pickle', 'rb') as f:
    tf = pickle.load(f)
    
with open('../data/tfidf_lda_tf_vectorizer_ignore_keywords.pickle', 'rb') as f:
    tf_vectorizer = pickle.load(f)

In [ ]:
pyLDAvis.sklearn.prepare(model, tf, tf_vectorizer)

## Measuring the Efficacy of Topic Models (WRITEUP: WHO?)

Main question: How do we evaluate how well a topic (from LDA for example) represents a meaningful "topic" or theme?

TODO: do some research on this??? There must be some papers etc that try to formalise this that we can borrow ideas from?

Paper dump:
  - Looks like a good summary paper: http://www.aclweb.org/anthology/E14-4005 Find more papers from this ones references?
    - " KL-divergence (Li and McCallum, 2006; Wang et al., 2009; Newman et al., 2009), cosine measure (He et al., 2009; Ramage et al., 2009) and the average Log Odds Ratio (Chaney and Blei, 2012). "
    - "Kim and Oh (2011) also applied  the  cosine  measure  and  KL-Divergence which were compared with four other measures: Jaccard’s Coefficient, Kendall’s τ coefficient, Discount  Cumulative  Gain  and  Jensen  Shannon  Divergence (JSD)."
  - Cool name haven't read it: http://papers.nips.cc/paper/3700-reading-tea-leaves-how-humans-interpret-topic-models.pdf
  
We considered all of these metrics, and found th Jaccard Index to be most suitable. This was primarily due to it's use of set operations, which are invariant to ordering and number of observations.

### Reading Tea Leaves Paper
http://papers.nips.cc/paper/3700-reading-tea-leaves-how-humans-interpret-topic-models.pdf

### Word Overlap

I think this is used as a baseline measure in the summary paper above (http://www.aclweb.org/anthology/E14-4005). Should be a quick implementation so worth a try.

### Jaccard Index

From the papers above this seems to have been used relatively often for _linking machine-generated topics to human topics_ and so maybe this is a good application for it. Apparently explored here "https://link.springer.com/chapter/10.1007/978-3-642-19437-5_13" but I haven't read it.


In [ ]:
def jaccard_index(a, b):
    a = set(a)
    b = set(b)
    
    num_shared = float(len(a & b))
    num_total = float(len(a | b))
    
    return num_shared/num_total

Consider the group of language keywords as the best possible topic for each language. Compare each of our machine generated topics with each of our ideal topics by computing their Jaccard Index:

Consider the group of language keywords as the best possible topic for each language. Compare each of our machine generated topics with each of our ideal topics by computing their Jaccard Index:


In [ ]:
topic1_keywords = [

"this",
"function",
"if",
"the",
"return",
"var",
"to",
"for",
"is",
"true",
"in",
"of",
"value",
"data",
"null",
"length",
"and",
"else",
"false",
"name",
"type",
"new",
"const",
"it",
"assert",
"object",
"be",
"options",
"key",
"that"]

In [ ]:
topic3_keywords = [
"self",
"def",
"import",
"from",
"none",
"user",
"in",
"ndef",
"response",
"name",
"assert",
"equal",
"id",
"not",
"str",
"models",
"true",
"request",
"get",
"email",
"assert",
"realm",
"data",
"message",
"result",
"for",
"nclass",
"dict",
"django",
"thread",
"url"]

In [ ]:
topic4_keywords = [
"react",
"from",
"default",
"nimport",
"nexport",
"createsvgicon",
"path",
"import",
"fragment",
"xd0",
"xe0",
"classname",
"as",
"props",
"utils",
"none",
"div",
"proptypes",
"createelement",
"theme",
"xe1",
"material",
"xd1",
"button",
"classes",
"m0",
"fill",
"ui",
"xe2",
"0h24v24h0v0z"]

In [ ]:
print(jaccard_index(javascript_keywords, topic1_keywords),
jaccard_index(javascript_keywords, topic3_keywords),
jaccard_index(javascript_keywords, topic4_keywords))

In [ ]:
print(jaccard_index(r_keywords, topic1_keywords),
jaccard_index(r_keywords, topic3_keywords),
jaccard_index(r_keywords, topic4_keywords))

In [ ]:
print(jaccard_index(python_keywords, topic1_keywords),
jaccard_index(python_keywords, topic3_keywords),
jaccard_index(python_keywords, topic4_keywords))

### Kendall’s τ Coefficient

Measures the association between two ranked lists. Source: Computational Linguistics and Intelligent Text Processing book.

### Evaluating Topic Models
TODO: better title needed

Idea:
  - save the actual % of each program langauge per repo
  - Then try to use LDA model to tell us "I believe repo <x> is 10% Topic 1, 20% Topic 2 etc". 
  - Use analysis from above two sections to create a "most likely mapping from lda topic to programming language".
  - rate our models

Here we can do cross-validation etc.

Load in the test data set:

In [ ]:
test_dataset = pandas.read_csv("../data/test-dataset.csv.gz", header=None, names=['repo', 'language', 'topics', 'documents'])

# Remove Github 'topics' since we don't use them in this analysis
test_dataset = test_dataset.drop(columns='topics')

test_dataset.head()

To evaluate the mixture model, we must label each repository with it's percentage of each programming language:

In [ ]:
def calculate_language_percentages(group):
    total_python_length = 0
    total_r_length = 0
    total_javascript_length = 0
    
    for index, repo, language, document in group.itertuples():
        if language == 'python':
            total_python_length += len(document)
            
        if language == 'javascript':
            total_javascript_length += len(document)
            
        if language == 'r':
            total_r_length += len(document)
            
    total_length = total_python_length + total_r_length + total_javascript_length
            
    return pandas.Series([
        total_python_length/total_length,
        total_r_length/total_length,
        total_javascript_length/total_length,
    ])

In [ ]:
test_composition_actual = test_dataset.groupby(by='repo').apply(calculate_language_percentages)
test_composition_actual.columns = ['python', 'r', 'javascript']

Here are the programming language percentages for each of repository in our test dataset:

In [ ]:
test_composition_actual

In [ ]:
combined_test_documents = test_dataset.groupby(by='repo').apply(concat_texts)

In [ ]:
with open('../data/concatDocs_lda_model.pickle', 'rb') as f:
    model = pickle.load(f)
    
with open('../data/concatDocs_tf_train.pickle', 'rb') as f:
    tf_train = pickle.load(f)
    
with open('../data/concatDocs_tf_vectorizer.pickle', 'rb') as f:
    tf_vectorizer = pickle.load(f)

with open('../data/concatDocs_tf_test.pickle', 'rb') as f:
    tf_test = pickle.load(f)

In [ ]:
combined_test_documents_tf = tf_vectorizer.transform(combined_test_documents)
combined_test_model = model.transform(combined_test_documents_tf)

In [ ]:
combined_test_model

In [ ]:
# insert kish's mappings here


### KL Divergence

In [ ]:
test_composition_estim=test_composition_actual

In [ ]:
if all(test_composition_estim.index==test_composition_actual.index)==True:
    KDDataframe=pandas.DataFrame([],columns=["Repo","KD Divergence"])
    for i in range(0,len(test_composition_actual)):
        a=0
        for j in range(0,len(test_composition_actual.iloc[i])):
            a=a-((test_composition_actual.iloc[i][j])*math.log(test_composition_actual.iloc[i][j]/test_composition_estim.iloc[i][j]))
        #m=pandas.DataFrame([test_composition_actual.index[i],a],columns=["Repo","KD Divergence"])
        print(a)
        #KDDataframe.append(m)
#KDDataframe

# Notes:
  - our keyword lists have simliar/common words e.g. python javascript and r all share some keywords. this can be seen in the documents. choosing another language, with a completely different set of keywords might prove easier to differentiate for the LDA model. somethind somthing LDA uses distance, but if true topics share key words, then distance metric breaks.